In [1]:
from alchzh_aoc_helpers.nb import AOCWidget
from alchzh_aoc_helpers.aliases import *
A = AOCWidget(2023, 5); A

Output()

In [2]:
seeds = [int(s) for s in A.L[0].split()[1:]]
ranges = [[seeds[s], seeds[s] + seeds[s+1]] for s in range(0, len(seeds), 2)]

maps = [
    [
        [int(i) for i in l.split()]
        for l in m.split("\n")[1:]
    ]
    for m in str(A).strip().split("\n\n")[1:]
]

def smap(s, maps=maps):
    v = s
    for map in maps:
        for j, i, k in map:
            if 0 <= v - i < k:
                v = j + v - i
                break
    return v


def rmap(rng, maps=maps):
    v = [rng]
    for map in maps:
        w = []
        while len(v):
            a, b = v.pop(0)
            o = len(w)
            for j, i, k in map:
                if b - a == 0:
                    continue
                p = j - i
                if i <= a < i + k:
                    if i + k < b:
                        v.append((i + k, b))
                        w.append((a + p, i + k + p))
                    else:
                        w.append((a + p, b + p))
                elif i < b <= i + k:
                    v.append((a, i))
                    w.append((i + p, b + p))
                elif a < i and b > i + k:
                    v.append((a, i))
                    v.append((i + k, b))
                    w.append((i + p, i + k + p))
            if len(w) == o:
                w.append((a, b))
        v = []
        c = None
        for a, b in sorted(w):
            if c is None or a > c[1]:
                c = [a, b]
                v.append(c)
            else:
                c[1] = b            
    return min(v)[0]

print("Part 1:", min(smap(s) for s in seeds))
print("Part 2:", min(rmap(r) for r in ranges))

Part 1: 227653707
Part 2: 78775051
